# Amazon Bedrock boto3 セットアップ

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

---

このデモノートブックでは、[`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) を使用して [Amazon Bedrock](https://aws.amazon.com/bedrock/) の基盤モデルと連携する方法を示します。

---

## 前提条件

⚠️ すでに [../download-dependencies.sh スクリプト](../download-dependencies.sh) を実行して、Bedrock の使用に必要な SDK パッケージを取得しているはずです。

パッケージの準備ができたら、以下のセルを実行してノートブックカーネルにインストールしてください:

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

このノートブックでは、AWS SDK を使用して Bedrock モデルを直接呼び出すデモを行いますが、ワークショップ後のノートブックでは、 [LangChain](https://github.com/hwchase17/langchain) もインストールする必要があります:

In [ ]:
%pip install --quiet langchain==0.0.249

---

## boto3 クライアントの作成

Bedrock API との通信は AWS SDK for Python を介して行われます: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html).


ご利用中の環境によっては、Bedrock サービスクライアントを作成する際、設定のカスタマイズの必要性が生じる場合があります。 そこで、さまざまなオプションを渡すことができる `get_bedrock_client()` ユーティリティメソッドを用意しました。実装は [../utils/bedrock.py](../utils/bedrock.py) をご参照ください。


#### デフォルトの証明書チェーンを使用

このノートブックを [Amazon Sagemaker Studio](https://aws.amazon.com/sagemaker/studio/) から実行しており、Sagemaker Studio [実行ロール](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) が Bedrock アクセスできる権限を持っている場合、以下のセルをそのまま実行できます。デフォルトの AWS 認証情報が Bedrock にアクセスできるコンピュータからノートブックを実行している場合も同様です。

#### 別のリージョンを使用

このノートブックを自分のコンピュータや Bedrock が設定されている AWS リージョンとは別の AWS リージョンに存在する SageMakerノートブックから実行している場合、以下の `os.environ['AWS_DEFAULT_REGION']` 行のコメントを外して、使用するプロファイルを指定してください。

#### 特定のプロファイルを使用

このノートブックを自分のコンピュータから実行している場合、AWS CLI を複数のプロファイルで設定していて、Bedrock にアクセスできるプロファイルがデフォルトのプロファイルでない場合、以下の `os.environ['AWS_PROFILE']` 行のコメントを外して、使用するプロファイルを指定してください。

#### 別のロールを使用

Bedrock にアクセスするために、特定の別の [IAM ロール](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) を設定している場合、以下の `os.environ['BEDROCK_ASSUME_ROLE']` 行のコメントを外して、指定することができます。現在のユーザーまたはロールがそのようなロールを[引き受ける](https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRole.html)権限を持っていることを確認してください。

#### カスタムのサービスエンドポイント URL を使用

カスタムの [サービスエンドポイント URL](https://docs.aws.amazon.com/general/latest/gr/rande.html) を使用してプレビューの一部として Bedrock にアクセスする必要がある場合 `os.environ['BEDROCK_ENDPOINT_URL']` 行のコメントを外して編集する必要があります。

#### `langchain` についてのメモ

`langchain` が提供する Bedrock クラスは、デフォルトで Bedrock boto3 クライアントを作成します。Bedrock の設定をカスタマイズするには、以下でメソッドを明示的に bedrock クライアントを作成し、[`langchain.Bedrock`](https://python.langchain.com/docs/integrations/llms/bedrock) クラスのインスタンス生成メソッドに `client=boto3_bedrock` として渡すことを推奨します。

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ コメントを外して、AWS の設定に応じて以下の行を編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

#### 接続の検証

`list_foundation_models()` メソッドを試すことで、クライアントが動作することを確認できます。

In [ ]:
boto3_bedrock.list_foundation_models()

---

## `InvokeModel` のボディと出力

Amazon Bedrock クライアントの `invoke_model()` メソッド (`InvokeModel` API) は、どのモデルを使用する場合でも、テキスト生成と処理タスクのほとんどに使用する主要なメソッドです。

このメソッドは共有されますが、入出力のフォーマットは使用する基盤モデルによって異なります:

### Amazon Titan Large

#### 入力
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### 出力

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### AI21 Jurassic (Grande and Jumbo) 

#### 入力

```json
{
    "prompt": "<prompt>",
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5,
    "stopSequences": [],
    "countPenalty": {"scale": 0},
    "presencePenalty": {"scale": 0},
    "frequencyPenalty": {"scale": 0}
}
```

#### 出力

```json
{
    "id": 1234,
    "prompt": {
        "text": "<prompt>",
        "tokens": [
            {
                "generatedToken": {
                    "token": "\u2581who\u2581is",
                    "logprob": -12.980147361755371,
                    "raw_logprob": -12.980147361755371
                },
                "topTokens": null,
                "textRange": {"start": 0, "end": 6}
            },
            //...
        ]
    },
    "completions": [
        {
            "data": {
                "text": "<output>",
                "tokens": [
                    {
                        "generatedToken": {
                            "token": "<|newline|>",
                            "logprob": 0.0,
                            "raw_logprob": -0.01293118204921484
                        },
                        "topTokens": null,
                        "textRange": {"start": 0, "end": 1}
                    },
                    //...
                ]
            },
            "finishReason": {"reason": "endoftext"}
        }
    ]
}
```

### Anthropic Claude

#### 入力

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### 出力

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Stability AI Stable Diffusion XL

#### 入力

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### 出力

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

---

## 一般的な推論パラメータ定義

### ランダム性と多様性

基盤モデルは、レスポンスのランダム性と多様性を制御するために以下のパラメータをサポートしています。

**Temperature** – 大規模言語モデルでは、確率を使用して単語を順番に構成します。任意の次の単語を選択する確率分布があります。
Temperature をゼロに近づけると、モデルは確率の高い単語を選択する傾向があります。
Temparature を大きくすると、モデルは確率の低い単語を選択する傾向があります。

技術的な説明をすると、Tempareture によるサンプリング手法の実装では、Tempareture によって次のトークンを取得する確率密度関数が調整されます。
このパラメータは、密度関数の曲線を深くしたり平坦にしたりできます。値が小さいほど曲線が急となり応答がより決定論的になります。
値が大きいほど曲線が平坦となりランダムな応答が多くなります。

**Top K** – Temperature は潜在的な単語の確率分布を定義し、Top K は単語を選択しなくなった時のカットオフを定義します。
例えば、K = 50 の場合、モデルは特定のシーケンスで次にくる可能性が最も高い 50 の単語から選択します。
これにより、シーケンスの次に珍しい単語が選択される可能性が低くなります。
技術的な説明をすると、Top K は、Top-K-filtering のために保持する、最も確率の高い語彙の数です。これにより、確率の高いトークンの分布が制限されるため、
モデルは最も確率の高いトークンを１つ選択します。

**Top P** – Top P は、潜在的な選択肢の確率の合計に基づいてカットオフを定義します。
Top P を 1.0 未満に設定すると、モデルは最も可能性の高いオプションを考慮し、可能性の低いオプションを無視します。 
Top P は Top K と似ていますが、選択肢の数に上限を設けるのではなく、その確率の合計に基づいて選択肢を制限します。
例えば、"I hear the hoof beats of ," というサンプルプロンプトでは、モデルに次の単語として、
"horses,"、"zebras" または "unicorns" を提供するようにモデルを設定することができます。
Top K や Top P に上限を設定せずに Temperature を制限せずに温度を最大に設定すると、"unicorns" のような珍しい結果が得られる確率が高くなります。
Temperature を 0 に設定すると、"horses" の確率が高くなります。Temperature を高く設定し、Top K または Top P を最大に設定すると、
"horses" の確率が高くなり、"unicorns" の確率が低くなります。

### 長さ

以下のパラメータは、生成されるレスポンスの長さを制御します。

**レスポンスの長さ** – 生成されたレスポンスで使用するトークンの最小数と最大数を設定します。

**長さペナルティ** – 長さペナルティは、より長い回答にペナルティを科すことで、モデルをより簡潔な出力に最適化します。
長さペナルティはレスポンスの長さとは異なり、最小または最大のレスポンスの長さをハードカットします。

技術的な説明をすると、長さのペナルティは、長い回答に対して指数関数的にモデルにペナルティを与えます。0.0 はペナルティなしを意味します。
長いシーケンスを生成するモデルには 0.0 より小さい値を設定し、短いシーケンスを生成するモデルには 0.0 より大きな値を設定します。

### 繰り返し

次のパラメータは、生成されたレスポンスの繰り返しをコントロールするのに役立ちます。

**繰り返しペナルティ (存在ペナルティ)** – 回答における同じ単語の (トークン) の繰り返しを防ぎます。
1.0 はペナルティなしを意味します。1.0 より大きな値で繰り返しが減少します。

---

## モデルを試す

理論的なことはさておき、実際のモデルを見てみましょう! 以下のセルを実行すると、各モデルの基本的な同期呼び出し例が表示されます:

### Amazon Titan Large

In [ ]:
# 独自のプロンプトを試したい場合は、このパラメータを編集してください!
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""

In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-tg1-large"
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("results")[0].get("outputText"))

### Anthropic Claude

In [ ]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # モデルプロバイダーとは異なるバーションを使用する場合は変更してください
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completion"))

### AI21 Jurassic Grande

In [ ]:
body = json.dumps({"prompt": prompt_data, "maxTokens": 200})
modelId = "ai21.j2-grande-instruct"  # モデルプロバイダーとは異なるバーションを使用する場合は変更してください
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("completions")[0].get("data").get("text"))

### Stability Stable Diffusion XL

In [ ]:
prompt_data = "a fine image of an astronaut riding a horse on Mars"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl"
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

**注意:** 出力は画像データを [base64 エンコード](https://docs.python.org/3/library/base64.html) した文字列です。 以下の例のように、任意の画像処理ライブラリ ([Pillow](https://pillow.readthedocs.io/en/stable/) など) を使用して画像をデコードすることができます。

```python
import base64
import io
from PIL import Image

base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
```

## ストリーミング出力の生成

大規模言語モデルの場合、長い出力シーケンスを生成するのにかなりの時間がかかることがあります。レスポンス全体が利用可能になるまで待つのではなく、遅延の影響を受けやすいアプリケーションでは、レスポンスをユーザーに**ストリーミング**することを好むかもしれません。

以下のコードを実行して、Bedrock の `invoke_model_with_response_stream()` メソッドで、レスポンスボディを別々のチャンクで返すことを実現する方法を確認してください。

In [ ]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-tg1-large"  # (こちらとリクエストボディを変更して、別のモデルを試してください)
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model_with_response_stream(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
stream = response.get('body')
output = []

if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))

## 埋め込みの生成

テキスト埋め込み (embedding) を使用して、テキストを意味のあるベクトル表現に変換します。 テキストの本文を入力すると、出力は (1 x n) ベクトルになります。
埋め込みベクトルはさまざまな用途に使用できます。bedrock は現在、テキストの類似性 (テキストの本文間のセマンティックな類似性の発見) とテキストの取得 (検索など) をサポートするテキスト埋め込み用の１つのモデルを提供しています。
テキスト埋め込みモデルでは、入力テキストサイズは 512 トークンで、出力ベクトル長は 4096 です。
テキスト埋め込みモデルを使用するには、InvokeModel API オペレーションもしくは Python SDK を使用してください。
InvokeModel を使用して、指定されたモデルから入力テキストのベクトル表現を取得します。

現時点では、API を介した埋め込みモデルとして `amazon.titan-e1t-medium` のみ利用できます。

#### 入力

```json
{
    "inputText": "<text>"
}
```

#### 出力

```json
{
    "embedding": []
}
```


テキストの埋め込みを生成する方法を見てみましょう:

In [ ]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."

In [ ]:
body = json.dumps({"inputText": prompt_data})
modelId = "amazon.titan-e1t-medium" 
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

embedding = response_body.get("embedding")
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

## 次のステップ

このノートブックでは、AWS Python SDK を使って Amazon Bedrock のモデルを呼び出す基本的な例を幾つか紹介しました。これで、他のラボで様々なユースケースやパターンを深く掘り下げる準備が整いました。